In [1]:
import torch
import torch.nn as nn
from transformers import ViTModel, BertModel, Wav2Vec2Model
from torch.nn.functional import normalize

class AdaptiveModalityEncoder(nn.Module):
    def __init__(self, input_dim=768):
        super(AdaptiveModalityEncoder, self).__init__()
        self.transformer_block = nn.TransformerEncoderLayer(d_model=input_dim, nhead=8)
        self.gate = nn.Linear(input_dim, 1)  # Gating mechanism to assign relevance to each modality
    
    def forward(self, modality_features):
        # modality_features is a list of tensors (img_features, text_features, audio_features)
        stacked_features = torch.stack(modality_features, dim=1)
        transformed = self.transformer_block(stacked_features)  # (batch, modalities, dim)
        gated_weights = torch.softmax(self.gate(transformed).squeeze(-1), dim=1)
        weighted_features = (gated_weights.unsqueeze(-1) * transformed).sum(dim=1)
        return weighted_features, gated_weights

class HierarchicalCrossModalityAttention(nn.Module):
    def __init__(self, input_dim=768):
        super(HierarchicalCrossModalityAttention, self).__init__()
        self.cross_attention1 = nn.MultiheadAttention(embed_dim=input_dim, num_heads=8)
        self.cross_attention2 = nn.MultiheadAttention(embed_dim=input_dim, num_heads=8)
    
    def forward(self, img_features, text_features, audio_features):
        # First level of cross-modality attention with text as the pivot modality
        text_img_attn, _ = self.cross_attention1(text_features, img_features, img_features)
        text_audio_attn, _ = self.cross_attention1(text_features, audio_features, audio_features)
        fused_text = (text_img_attn + text_audio_attn) / 2
        
        # Second level of cross-modality attention with image as the pivot modality
        img_text_attn, _ = self.cross_attention2(img_features, fused_text, fused_text)
        img_audio_attn, _ = self.cross_attention2(img_features, audio_features, audio_features)
        fused_img = (img_text_attn + img_audio_attn) / 2
        
        # Final fusion
        fused_features = (fused_text + fused_img + audio_features) / 3
        return fused_features

class MACAST(nn.Module):
    def __init__(self):
        super(MACAST, self).__init__()
        
        # Modality encoders
        self.image_encoder = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
        self.text_encoder = BertModel.from_pretrained("bert-base-uncased")
        self.audio_encoder = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        
        # Adaptive Modality Encoder
        self.adaptive_modality_encoder = AdaptiveModalityEncoder()
        
        # Hierarchical Cross-Modality Attention
        self.hierarchical_attention = HierarchicalCrossModalityAttention()
        
        # Sentiment classification head
        self.classification_head = nn.Linear(768, 3)  # 3 sentiment classes
        
        # Auxiliary losses for self-supervised modality contrastive alignment
        self.contrastive_temp = 0.07
        self.contrastive_loss_fn = nn.CrossEntropyLoss()
    
    def forward(self, image, text, audio):
        # Encode each modality
        img_features = self.image_encoder(pixel_values=image).last_hidden_state[:, 0]
        text_features = self.text_encoder(input_ids=text["input_ids"], attention_mask=text["attention_mask"]).last_hidden_state[:, 0]
        audio_features = self.audio_encoder(input_values=audio["input_values"]).last_hidden_state[:, 0]
        
        # Adaptive Modality Encoding
        weighted_features, gated_weights = self.adaptive_modality_encoder([img_features, text_features, audio_features])
        
        # Hierarchical Cross-Modality Attention Fusion
        fused_features = self.hierarchical_attention(img_features, text_features, audio_features)
        
        # Classification head
        logits = self.classification_head(fused_features)
        
        # Contrastive alignment for modality embeddings
        img_proj, text_proj, audio_proj = map(normalize, [img_features, text_features, audio_features])
        contrastive_loss = self.modality_contrastive_loss(img_proj, text_proj, audio_proj)
        
        return logits, contrastive_loss

    def modality_contrastive_loss(self, img_proj, text_proj, audio_proj):
        # Implementing contrastive loss for cross-modal alignment
        batch_size = img_proj.size(0)
        labels = torch.arange(batch_size).to(img_proj.device)
        
        logits_img_text = torch.mm(img_proj, text_proj.T) / self.contrastive_temp
        logits_img_audio = torch.mm(img_proj, audio_proj.T) / self.contrastive_temp
        logits_text_audio = torch.mm(text_proj, audio_proj.T) / self.contrastive_temp
        
        loss_img_text = self.contrastive_loss_fn(logits_img_text, labels)
        loss_img_audio = self.contrastive_loss_fn(logits_img_audio, labels)
        loss_text_audio = self.contrastive_loss_fn(logits_text_audio, labels)
        
        return (loss_img_text + loss_img_audio + loss_text_audio) / 3

# --- Training Loop ---
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for images, texts, audios, labels in dataloader:
            optimizer.zero_grad()
            
            # Forward pass
            logits, contrastive_loss = model(images, texts, audios)
            cls_loss = criterion(logits, labels)
            total_loss = cls_loss + contrastive_loss  # Joint loss
            
            # Backward pass and optimization
            total_loss.backward()
            optimizer.step()
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss.item():.4f}")

/Users/ayush/Desktop/ayush/UvA-multi-modal-ai/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ayush/Desktop/ayush/UvA-multi-modal-ai/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
